In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
!pip install konlpy
!pip install mecab-python3
!export MECAB_HOME=/usr/local/lib/mecab/dic/mecab-ko-dic
!pip install eunjeon

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 11.33 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 43.7 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-03-21 04:03:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c3:9b0a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP

In [ ]:
!apt-get update -qq
!apt-get install -y fonts-nanum
!apt-get install -y g++ openjdk-8-jdk python-dev python3-dev
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np

def make_dic(texts):
  words = []
  m = Mecab()

  for snt in texts:
    res = m.pos(snt)
    #형태소 분석을 수행해서 품사를 부착해주는 메서드가 m.pos() -> (pos는 part of speech)
    for(lex, pos) in res:
      if pos[0] == 'N' or pos[0]=='V':
        #pos[0]인 첫 번째 순서의 형태소의 품사가 N인지 V인지 체크 ==> 내용어인 contents word를 뽑기
        words.append(lex)

  #단어의 빈도수 계산
  dic = FreqDist(np.hstack(words))
  #나 : 10, 학교 : 15 형태로 형성
  print("사전 크기 : {0:d}".format(len(dic)))

  #인덱스를 저장할 변수 초기화
  indexes = {}
  words = {}

  #단어에 고유 번호인 인덱스를 부여
  for num, word in enumerate(dic):
    #dic에서 enumerate()로 딕셔너리에 순서를 매겨 인덱스인 역할인 num과 word를 갖고 옴
    #실제 dic은 {word:idx}로 되어 있음
    idx = num+2
    indexes[word] = idx
    words[idx] = word

  #문장 길이 정규화에 사용한 패딩 인덱스
  indexes['pad']=1
  words[1]='pad'
  #사전에 없는 단어에 대한 인덱스
  indexes['unk']=0
  words[0]='unk'

  return (indexes, words)

def word2index(indexes, word):
  idx = indexes[word] if word in indexes else indexes['unk']
  return idx

def index2word(words, index):
  w = words[index] if index in words else -1
  return w

In [ ]:
texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."
         ]

(indexes, words) = make_dic(texts)
print(word2index(indexes, "하늘"), word2index(indexes, "학교"))
#글자로 인덱스 매핑, 학교같은 없는 단어는 0번 인덱스에 넣기

print(index2word(words, 4), index2word(words, 0))
#인덱스로 글자 매핑, 없는 인덱스는 unk로 unknown word로 out of vocab, oov라고 함
#물론 fastText로 n-gram 방식을 쓸 수 있는데
#이보다 더 발전된 방법은 word piece를 구성하는 방법인 BPE(Byte Pair Encoding) 방법도 존재. 이를 이용한 게 BERT

사전 크기 : 24
7 0
나 unk


In [ ]:
from konlpy.tag import Mecab
import numpy as np
from gensim.models import Word2Vec, KeyedVectors

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."
         ]

#Word2Vec 학습에 사용할 데이터 만들기
m = Mecab()
result = []

for sent in texts:
  tag = m.pos(sent)
  words = []
  for (lex, pos) in tag:
    if pos[0]=='N':
      #audtkfb(cpdjsaks cncnf)
      words.append(lex)
  result.append(words)

print(result, "\n")



[['날', '하늘', '점', '부끄럼'], ['잎새', '바람', '나'], ['별', '노래', '마음', '것', '사랑'], ['나', '길'], ['밤', '별', '바람']] 



In [ ]:
#Word2Vec 학습시키기
model = Word2Vec(sentences=result, vector_size=10, window=1, min_count=1, workers=1, sg=0)
#result를 input으로 넣어서 size가 10차원인 임베딩(projection)을 해서 붙은 w를 학습, window는 앞뒤로 1글자씩
#size : 임베딩 차원
#window : 컨텍스트 윈도우 크기
#min_count : min_count: 모델이 학습할 때 고려할 최소 단어 빈도. min_count=5로 설정하면 코퍼스에 5번 미만 등장한 단어는 무시
#workers : 학습에 사용할 CPU 코어의 수. 이 값을 높이면 학습 속도가 빨라지지만 너무 높이면 메모리 소비 커
#sg : 학습 알고리즘을 선택하는 파라미터로 sg=0이면 CBOW가 사용되고, sg=1이면 Skip-gram이 사용.
#CBOW는 주변 단어를 사용하여 중심 단어를 예측하는 반면, Skip-gram은 중심 단어를 사용하여 주변 단어를 예측.
#Skip-gram은 희소한 데이터셋에서 더 좋은 결과를 보일 수 있지만, CBOW는 학습 속도가 빠르다는 장점

#값 읽어오기
print(model.wv['하늘'])
#'하늘'과 유사한 단어 가져오기 이 때 코사인 유사도 값이 같이 나옴
print(model.wv.most_similar("하늘"),"\n")

#Word2Vec 모델 저장하기
model.wv.save_word2vec_format('경로')

#Word2Vec 모델 로드하기
loaded_model = KeyedVectors.load_word2vec_format('경로')

#값 읽어오기
print(model.wv['하늘'])
#'하늘'과 유사한 단어 가져오기 이 때 코사인 유사도 값이 같이 나옴
print(model.wv.most_similar("하늘"),"\n")

[ 0.02348376 -0.04519032  0.08388732 -0.09858163  0.06764641  0.02914417
 -0.04932832  0.04398187 -0.01739575  0.06711384]
[('잎새', 0.5111488699913025), ('날', 0.43408337235450745), ('것', 0.3862057030200958), ('바람', 0.3002755343914032), ('길', 0.141779825091362), ('밤', 0.057554230093955994), ('마음', -0.027840567752718925), ('사랑', -0.05887392535805702), ('부끄럼', -0.09835173189640045), ('점', -0.10117148607969284)] 

[ 0.02348376 -0.04519032  0.08388732 -0.09858163  0.06764641  0.02914417
 -0.04932832  0.04398187 -0.01739575  0.06711384]
[('잎새', 0.5111488699913025), ('날', 0.43408337235450745), ('것', 0.3862057030200958), ('바람', 0.3002755343914032), ('길', 0.141779825091362), ('밤', 0.057554230093955994), ('마음', -0.027840567752718925), ('사랑', -0.05887392535805702), ('부끄럼', -0.09835173189640045), ('점', -0.10117148607969284)] 



In [ ]:
#GloVe
!pip install glove-python-binary

ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary


In [ ]:
import urllib.request
import zipfile
import os
from gensim.models import KeyedVectors

# GloVe 모델 다운로드
glove_vectors_file = 'glove.6B.100d.txt'
glove_vectors_url = 'https://nlp.stanford.edu/data/glove.6B.zip'
if not os.path.exists(glove_vectors_file):
    urllib.request.urlretrieve(glove_vectors_url, 'glove.6B.zip')
    with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
        zip_ref.extractall('.')

# GloVe 모델 로드
glove_model = KeyedVectors.load_word2vec_format(glove_vectors_file, binary=False)

# 모델 사용 예시
vector = glove_model['word']
print(vector)


ValueError: invalid literal for int() with base 10: 'the'

In [ ]:
#Co Occurrence Matrix 생성

corpus = Corpus()
corpus.fit(result, window=1)
#

#GlovVe  학습시키기
glove=Glove(no_components=10, learning_rate = 0.05)
#10차원 임베딩
glove.fit(corpus.matrix, epochs-20, no_threads=1, verbose=True)
glove.add_dictionary(corpus.dictionary)

#값 읽어오기
print(model.wv['하늘'])
#'하늘'과 유사한 단어 가져오기 이 때 코사인 유사도 값이 같이 나옴
print(model.wv.most_similar("하늘"),"\n")


#Word2Vec 모델 저장하기
glove.save_word2vec_format('경로')
#Word2Vec 모델 로드하기
loaded_model = glove.load('경로')

#값 읽어오기
print(model.wv['하늘'])
#'하늘'과 유사한 단어 가져오기 이 때 코사인 유사도 값이 같이 나옴
print(model.wv.most_similar("하늘"),"\n")

NameError: name 'Corpus' is not defined

In [ ]:
"""
Embedding Layer : 단어가 들어왔을 때 lookup Table로부터 임베딩 벡터를 다이렉트로 연결할 수 있도록 하는 layer

임베딩 벡터를 사용하는 방법

- 사전 학습된 벡터를 사용
--> Word2Vec, GloVe 등을 읽어와서 사용하며 학습 과정에 업데이트 없음

- 학습 과정에서 벡터 업데이트
--> 사전 학습된 벡터 또는 임의의 벡터를 만들고 학습 과정에서 업데이트 => 파인 튜닝(fine-tuning)
--> 파이토치에서는 nn.Embedding()을 사용해서 구현

"""

In [ ]:
#파이토치에서 임베딩 레이어를 이용해서 사전 만들기

import torch
import torch.nn as nn

train_data = "I like deep learning I like NLP I enjoy flying"

#중복 제거를 위해 set이용
word_set = set(train_data.split())

#단어 집합의 각 단어에 고유한 정수 매핑
vocab = {word: i+2 for i, word in enumerate(word_set)}
#2번부터 할당
vocab['unk']=0
vocab['pad']=1
print(vocab)


#임베딩 테이블 : (단어수, 임베딩 사이즈) ===>녹음 듣기
embedding_table = torch.FloatTensor([
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.1, 0.8, 0.3],
        [0.7, 0.8, 0.2],
        [0.1, 0.8, 0.7],
        [0.9, 0.2, 0.1],
        [0.1, 0.1, 0.9],
        [0.2, 0.1, 0.7],
        [0.3, 0.1, 0.1]
])

#입력문장
input_snt = "I like football".split()

#각 단어를 정수로 변환
idxes=[]

for word in input_snt:
  idx = vocab[word] if word in vocab else vocab['unk']
  idxes.append(idx)

idexes = torch.LongTensor(idxes)

#입력 문장의 임베딩 벡터 가져오기: ["I", "like", "football (unk)"]
lookup_result = embedding_table[idxes, :]
print(lookup_result)

#임베딩 층 만들기
embedding_layer = nn.Embedding(num_embeddings = len(vocab), embedding_dim=3, padding_idx=1)
print(embedding_layer.weight)

{'enjoy': 2, 'flying': 3, 'like': 4, 'deep': 5, 'learning': 6, 'I': 7, 'NLP': 8, 'unk': 0, 'pad': 1}
tensor([[0.2000, 0.1000, 0.7000],
        [0.1000, 0.8000, 0.7000],
        [0.0000, 0.0000, 0.0000]])
Parameter containing:
tensor([[ 0.5841, -0.3320,  0.2817],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2940,  0.2002,  0.3889],
        [-0.8157,  1.3281,  0.4583],
        [ 0.9548,  0.7584,  1.1593],
        [ 0.8501, -0.4825, -1.3522],
        [-0.8748, -0.2911, -1.2266],
        [ 1.1575, -2.4628,  1.1485],
        [-0.8116,  0.3236,  2.4082]], requires_grad=True)
